In [1]:
import pandas as pd
import random
import numpy as np
import copy

class Deck():
    def __init__(self, num_deck=1, is_black_jack = True):
        # num_decks = 0 means infinite number of decks and the pick up is with replacement
        self.contents = [(2, 'Diamonds'), (2, 'Hearts'), (2, 'Clubs'), (2, 'Spades'), (3, 'Diamonds'), (3, 'Hearts'),\
                         (3, 'Clubs'), (3, 'Spades'), (4, 'Diamonds'), (4, 'Hearts'), (4, 'Clubs'), (4, 'Spades'),\
                         (5, 'Diamonds'), (5, 'Hearts'), (5, 'Clubs'), (5, 'Spades'), (6, 'Diamonds'), (6, 'Hearts'),\
                         (6, 'Clubs'), (6, 'Spades'), (7, 'Diamonds'), (7, 'Hearts'), (2, 'Clubs'), '7 of Spades',\
                         (8, 'Diamonds'), (8, 'Hearts'), (8, 'Clubs'), (8, 'Spades'), (9, 'Diamonds'), (9, 'Hearts'),\
                         (9, 'Clubs'), (9, 'Spades'), (10, 'Diamonds'), (10, 'Hearts'), (10, 'Clubs'), (10, 'Spades'),\
                         (11, 'Diamonds'), (11, 'Hearts'), (11, 'Clubs'), (11, 'Spades'), (12, 'Diamonds'),\
                         (12, 'Hearts'), (12, 'Clubs'), (12, 'Spades'), (13, 'Diamonds'), (13, 'Hearts'),\
                         (13, 'Clubs'), (13, 'Spades'), (1, 'Diamonds'), (1, 'Hearts'), (1, 'Clubs'),\
                         (1, 'Spades')]
        if num_deck is not 0:
            self.decks = [self.contents for _ in range(num_deck)]
            self.decks = list(np.concatenate(self.decks, axis=0))
        else:
            self.decks = list(self.contents)
        self.num_deck = num_deck
        self.is_black_jack = is_black_jack

    def shuffle(self):
        # shuffle the cards
        random.shuffle(self.decks)
    
    def reset(self):
        self.decks = list(self.contents)
        random.shuffle(self.decks)

    def pop(self):
        # pop the top card out and hand it to the player
        card = self.decks.pop(0)
        if self.is_black_jack and int(card[0])>10:
            card = (10, card[1])
        # for num_deck = 0, pick up with replacement
        if self.num_deck == 0:
            self.reset()
        return card
    
card_possibility = {1:4/52, 2:4/52, 3:4/52, 4:4/52, 5:4/52, 6:4/52, 7:4/52, 8:4/52, 9:4/52, 10:16/52}

In [2]:
def total(hand):
    total = 0
    for card in hand:
        if card >=10:
            total+= 10
        elif card == 1:
            if total >= 11: 
                total+= 1
            else: 
                total+= 11
        else:
            total += card
    return total


def card_dic_2_possibility(card_dic):
    card_possibility = {1:4/52, 2:4/52, 3:4/52, 4:4/52, 5:4/52, 6:4/52, 7:4/52, 8:4/52, 9:4/52, 10:16/52, 12:4/52, 13:4/52}
    num_of_cards = sum(card_dic.values())
    for the_key in card_dic.keys():
#         if the_key>=10:
#             card_possibility[10]+= card_dic[the_key]/num_of_cards
#         else:
        card_possibility[the_key] = card_dic[the_key]/num_of_cards
    return card_possibility
        
    

def state_transition_pos(state, next_state, card_possibility):
    if next_state - state==11:
        return card_possibility['1']
    else:
        return card_possibility[next_state-state]
    
def explosion_pos(state, card_possibility):
    the_indexs = [the_key for the_key in card_possibility.keys() if the_key > (22 - state)]
    possibility = 0
    for the_key in the_indexs:
        possibility += card_possibility[the_key]
    return possibility
        
    
class agent():
    def __init__(self):
        self.hand=[]
        self.card_dic={1:4, 2:4, 3:4, 4:4, 5:4, 6:4, 7:4, 8:4, 9:4, 10:4, 11:4, 12:4, 13:4}
        
    def hit(self, card):
        self.card_dic[card[1]] -=1
        self.hand.append(card)
        return hand

In [3]:
def dealer_exploration_1card(state, next_state, card_possibility):
    if next_state - state > 11:
         return 0
    elif next_state - state==11:
        return card_possibility['1']
    else:
        return card_possibility[next_state-state]

def dealer_exploration_2card(state, next_state, card_dic):
    the_indexs = [the_key for the_key in card_dic.keys()]
    results = []
    sum_pos = 0
    for i in the_indexs:
        for ii in the_indexs:
            if i + ii == (next_state-state):
                results.append((i,ii))
    for combination in results:
        temp = copy.deepcopy(card_dic)
        if temp[combination[0]] > 1:
            temp[combination[0]] -= 1
            if temp[combination[1]] > 1:
                pre_card_possibility = card_dic_2_possibility(card_dic)
                card_possibility = card_dic_2_possibility(temp)
                sum_pos += pre_card_possibility[combination[0]] * card_possibility[combination[1]]
                
        
    
# def compute_exp_return

In [12]:
the_list = []
def dealer_tree(root, total_value, new_card,probability_father, probability, card_dic):
    if root + total_value >= 17:
        return_probability = copy.deepcopy(probability)
#         if new_card>10:
#             index = 10
#         else:
        index = new_card
        if root + total_value > 21:
            return_probability[-1] = probability_father
        else:
            return_probability[root + total_value-17] = probability_father
#         print(return_probability, probability_father, root + total_value)
        the_list.append(return_probability)
        return return_probability
    else:
        for i in range(1, 14):
            return_probability = copy.deepcopy(probability)
            temp_card_dic = copy.deepcopy(card_dic)
            if i < 10:
                index =i
                if i == 1 and root + total_value + 11 <= 21:
                    index = 11
            else:
                index = 10
            p = probability_father * card_dic_2_possibility(card_dic)[i]
#             print('p',p)
            temp_card_dic[i] -= 1 
            dealer_tree(root, total_value + index, i, p, return_probability,temp_card_dic)

In [13]:
for known in range(1,12):
    the_list = []
    full_deck = {1:4, 2:4, 3:4, 4:4, 5:4, 6:4, 7:4, 8:4, 9:4, 10:4, 11:4, 12:4, 13:4}
    full_deck[known] -= 1
    dealer_tree(known, 0 , 0, 1, [0, 0, 0, 0, 0, 0], full_deck)
    sum1=0
    sum2=0
    sum3=0
    sum4=0
    sum5=0
    sum6=0
    for i in range(len(the_list)):
        sum1 += the_list[i][0]
        sum2 += the_list[i][1]
        sum3 += the_list[i][2]
        sum4 += the_list[i][3]
        sum5 += the_list[i][4]
        sum6 += the_list[i][5]
    print(sum1, sum2, sum3, sum4, sum5, sum6)

0.13686463950085448 0.13682908636338428 0.12838842017234117 0.12492734458098267 0.16720358621088036 0.3057869231715582
0.13473832373192635 0.12731579456808595 0.1272707613455436 0.11929392965739376 0.11597212811067174 0.3754090625863796
0.12643738276854233 0.12657180965095952 0.11928607274643292 0.11885126743866382 0.11140841971210734 0.39744504768329647
0.1269431864804342 0.10999597787890625 0.11701622176097631 0.11205456544336158 0.11087950927876979 0.42311053915755337
0.1158896813007152 0.11926595396005142 0.11257121250092281 0.10049713278073259 0.10239210588317779 0.4493839135744026
0.16694766812059422 0.10645402905927319 0.10719186053213829 0.1007049316666462 0.09787847663196507 0.4208230339893839
0.3723448664078644 0.1385833815487343 0.07733443127482263 0.0788966658716112 0.07298698778514384 0.2598536671118236
0.13085746416975672 0.36298936571236573 0.12944463829318353 0.0682897637624276 0.06979142147145628 0.23862734659081034
0.12188622720133381 0.10392095413558577 0.35739125005

|  | $17 $ | $ 18 $ | $ 19 $ |$ 20 $ | $ 21 $ | $ >21 $ 
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | 
|$D$ = 1|0.13686463950085448 | 0.13682908636338428 | 0.12838842017234117 | 0.12492734458098267 | 0.16720358621088036 | 0.3057869231715582
|$D$ = 2| 0.13473832373192635 | 0.12731579456808595 | 0.1272707613455436 | 0.11929392965739376 | 0.11597212811067174 | 0.3754090625863796
|$D$ = 3| 0.12643738276854233 | 0.12657180965095952 | 0.11928607274643292 | 0.11885126743866382 | 0.11140841971210734 | 0.39744504768329647
|$D$ = 4| 0.1269431864804342 | 0.10999597787890625 | 0.11701622176097631 | 0.11205456544336158 | 0.11087950927876979 | 0.42311053915755337
|$D$ = 5| 0.1158896813007152 | 0.11926595396005142 | 0.11257121250092281 | 0.10049713278073259 | 0.10239210588317779 | 0.4493839135744026
|$D$ = 6| 0.16694766812059422 | 0.10645402905927319 | 0.10719186053213829 | 0.1007049316666462 | 0.09787847663196507 | 0.4208230339893839
|$D$ = 7| 0.3723448664078644 | 0.1385833815487343 | 0.07733443127482263 | 0.0788966658716112 | 0.07298698778514384 | 0.2598536671118236
|$D$ = 8| 0.13085746416975672 | 0.36298936571236573 | 0.12944463829318353 | 0.0682897637624276 | 0.06979142147145628 | 0.23862734659081034
|$D$ = 9| 0.12188622720133381 | 0.10392095413558577 | 0.35739125005537214 | 0.12225027908172217 | 0.06110882959412567 | 0.23344245993186055
|$D$ = 10| 0.1144181928090385 | 0.11287896102011571 | 0.11466240520259907 | 0.32887909897230555 | 0.11489750633525073 | 0.21426383566069046
|$D$ = 11| 0.1128628472665662 | 0.1144181928090385 | 0.11313103964990251 | 0.11472367670472444 | 0.32909742620452437 | 0.21576681736524406


In [88]:
sum([sum1, sum2, sum3, sum4, sum5, sum6])

1.0

### Zero Memory

|  | $17 $ | $ 18 $ | $ 19 $ |$ 20 $ | $ 21 $ | $ >21 $ 
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | 
|$D$ = 1|0.13853324706661213 | 0.13557625054021205 | 0.12959686209249507 | 0.12318045055860413 | 0.17165838918106313 | 0.3014548005610004
|$D$ = 2| 0.13389323624407973 | 0.12883065521759768 | 0.1260741786224704 | 0.12040621085972823 | 0.11433079465779937 | 0.3764649243983222
|$D$ = 3| 0.12837581599698272 | 0.12541720014374658 | 0.12044133296672851 | 0.11732714721750172 | 0.11163452039002349 |0.39680398328501354
|$D$ = 4| 0.12604716704099472 | 0.1119239390019091 | 0.11587961410428384 | 0.1125499175834215 | 0.10940280936099833 | 0.42419655290839475
|$D$ = 5| 0.11836031977506437 | 0.1181926216768421 | 0.11331006273683414 | 0.09917874602838665 | 0.10277064578182889 | 0.4481876040010462
|$D$ = 6| 0.16660540908326865 | 0.10674332959738149 | 0.10622833250808146 | 0.10138349222335077 | 0.09649745705994037 |0.42254197952797856
|$D$ = 7| 0.3738770616642109 | 0.13846232675833037 | 0.07823470011427316 | 0.07808496034928418 | 0.0736038707074383 | 0.2577370804064635
|$D$ = 8| 0.13081526038571548 | 0.36457150798279353 | 0.12927739033573496 | 0.06905922386884035 | 0.0689196862556933 | 0.23735693117122297
|$D$ = 9| 0.12338003684683549 | 0.1037721937095806 | 0.35867415449389434 | 0.12184216679685489 | 0.0617446175887824 | 0.23058683056405277
|$D$ = 10| 0.11440694325797474 | 0.11440694325797474 | 0.11440694325797474 | 0.33009321776777867 | 0.11440694325797483 | 0.21227900920032267
|$D$ = 11| 0.11440694325797474 | 0.11440694325797474 | 0.11440694325797474 | 0.11440694325797474 | 0.33009321776777867 | 0.21227900920032267



In [10]:
total_counter = 0
start_card = 6
theDeck = Deck()
for i in range(int(1e6)):
    sum_card = start_card
    theDeck.reset()
    while(sum_card<17): 
        the_card = theDeck.pop() 
        if int(the_card[0]) == 1 and sum_card + 11 <= 21:
            sum_card += 11
        else:
            sum_card += int(the_card[0])
    if sum_card == 17:
        total_counter += 1

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [11]:
total_counter

167951